In [29]:
# Version 1.7

# In this version the time in the simulation is change, now runs as fast as the cpu allows it
import pygame
import numpy as np
import sys
import random
from pygame.locals import *
import math
import os
# Parameters
WIDTH, HEIGHT = 1800, 1000
NUM_PEOPLE = 50 # This should be 50
PERSON_RADIUS = 5
MAX_SPEED = 1.75 #0.463 t
SEPARATION_RADIUS = 15
WALL_MARGIN = 15 # Separación entre las paredes
SPAWN_AREA = (40, 100, 900, 930)  # x_min, x_max, y_min, y_max
GROUP_SIZE = 1                   # Number of people per group
SPAWN_INTERVAL = 500 #3500            # Milliseconds between group spawns
INFECTION_RATE = 0.8  # % de las personas estarán infectadas
ASYMPTOMATIC_RATE = 0.5 # % de las personas serán asintómaticas

# Aleatoriedad de caminos, cantidad de middle path waypoint que una persona puede tener en un recorrido.
CASO_MENOR = 1
CASO_MAYOR = 6

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)  # For path waypoints
BLUE = (0, 0, 255)   # For passable walls
YELLOW = (255, 255, 0)

# Wait times
WALK_WAIT_RANGE = (1, 3)      # seconds
CHECKOUT_WAIT_RANGE = (4.0, 8.0) # seconds  ← tweak this


# Define the initial path waypoints 
'''These are the entries of the space that we are analyzing'''
initial_path_waypoints = [
    np.array([100, 800]),
    np.array([100, 100]),
]

# Define the final path waypoints 
'''These are the exits of the space that we are analyzing'''
final_path_waypoints = [
    np.array([350, 600]),
    np.array([700, 600]),
    np.array([1100, 600]),
    np.array([1600, 600]),
    ]

# Define path waypoints (green)
middle_path_waypoints = [# 975 is the midle point of the market
                         
    np.array([600, 70]),
    np.array([850, 70]),
    np.array([1100, 70]),
    np.array([1350, 70]),
    
    np.array([600, 100]),
    np.array([850, 100]),
    np.array([1100, 100]),
    np.array([1350, 100]),
    
    np.array([600, 130]),
    np.array([850, 130]),
    np.array([1100, 130]),
    np.array([1350, 130]),
    
    np.array([600, 170]),
    np.array([850, 170]),
    np.array([1100, 170]),
    np.array([1350, 170]),
    
    np.array([600, 200]),
    np.array([850, 200]),
    np.array([1100, 200]),
    np.array([1350, 200]),
    
    np.array([600, 230]),
    np.array([850, 230]),
    np.array([1100, 230]),
    np.array([1350, 230]),
    
    np.array([600, 270]),
    np.array([850, 270]),
    np.array([1100, 270]),
    np.array([1350, 270]),

    np.array([600, 300]),
    np.array([850, 300]),
    np.array([1100, 300]),
    np.array([1350, 300]),
    
    np.array([600, 330]),
    np.array([850, 330]),
    np.array([1100, 330]),
    np.array([1350, 330]),
    
    np.array([600, 370]),
    np.array([850, 370]),
    np.array([1100, 370]),
    np.array([1350, 370]),

    np.array([600, 400]),
    np.array([850, 400]),
    np.array([1100, 400]),
    np.array([1350, 400]),
    
    np.array([600, 430]),
    np.array([850, 430]),
    np.array([1100, 430]),
    np.array([1350, 430]),
    
    np.array([600, 470]),
    np.array([850, 470]),
    np.array([1100, 470]),
    np.array([1350, 470]),

    np.array([600, 500]),
    np.array([850, 500]),
    np.array([1100, 500]),
    np.array([1350, 500]),
    
    np.array([600, 530]),
    np.array([850, 530]),
    np.array([1100, 530]),
    np.array([1350, 530]),
    
    np.array([600, 570]),
    np.array([850, 570]),
    np.array([1100, 570]),
    np.array([1350, 570]),

    np.array([600, 600]),
    np.array([850, 600]),
    np.array([1100, 600]),
    np.array([1350, 600]),
]

checkout_map = {
    "F1": [
        [350, 650, -1], 
        [350, 700, -1], 
        [350, 750, -1],        
        [350, 800, -1],        
        [350, 850, -1],        
    ],
    "F2": [
        [470, 650, -1], 
        [470, 700, -1], 
        [470, 750, -1],        
        [470, 800, -1],        
        [470, 850, -1],        
    ],
    "F3": [
        [600, 650, -1], 
        [600, 700, -1], 
        [600, 750, -1],        
        [600, 800, -1],        
        [600, 850, -1],        
    ],
    "F4": [
        [720, 650, -1], 
        [720, 700, -1], 
        [720, 750, -1],        
        [720, 800, -1],        
        [720, 850, -1],        
    ],
    "F5": [
        [850, 650, -1], 
        [850, 700, -1], 
        [850, 750, -1],        
        [850, 800, -1],        
        [850, 850, -1],        
    ],
    "F6": [
        [970, 650, -1], 
        [970, 700, -1], 
        [970, 750, -1],        
        [970, 800, -1],        
        [970, 850, -1],        
    ],
    "F7": [
        [1100, 650, -1], 
        [1100, 700, -1], 
        [1100, 750, -1],        
        [1100, 800, -1],        
        [1100, 850, -1],        
    ],
    "F8": [
        [1220, 650, -1], 
        [1220, 700, -1], 
        [1220, 750, -1],        
        [1220, 800, -1],        
        [1220, 850, -1],        
    ],
    "F9": [
        [1350, 650, -1], 
        [1350, 700, -1], 
        [1350, 750, -1],        
        [1350, 800, -1],        
        [1350, 850, -1],        
    ],
    "F10": [
        [1470, 650, -1], 
        [1470, 700, -1], 
        [1470, 750, -1],        
        [1470, 800, -1],        
        [1470, 850, -1],        
    ],
    "F11": [
        [1600, 650, -1], 
        [1600, 700, -1], 
        [1600, 750, -1],        
        [1600, 800, -1],        
        [1600, 850, -1],        
    ],
    
}

# Define walls (black - solid, must avoid)
walls = [# A vector is define like this [x1, y1, x2, y2]
    # Screen boundaries
    [0, 0, WIDTH, 0],        # Top wall
    [WIDTH, 0, WIDTH, HEIGHT],  # Right wall
    [WIDTH, HEIGHT, 0, HEIGHT],  # Bottom wall
    [0, HEIGHT, 0, 0],        # Left wall
    
    # Interior walls with passages
    [200, 50, 1700, 50], # top wall market
    [1700, 50, 1700, 900], # right wall market
    [1550, 900, 200, 900], # botton wall market
    [200, 900, 200, 150], # left wall market
    
    # Pasillos
    [400, 150, 1500, 150], # division 1
    [400, 250, 1500, 250], # division 2
    [400, 350, 1500, 350], # division 3
    [400, 450, 1500, 450], # division 4
    [400, 550, 1500, 550], # division 5
]

# Functions
midle_value = 975 # half of the mall

# Pegamos toda la lista de puntos para dibujar en el mapa las trayectorias 
path_waypoints = initial_path_waypoints + middle_path_waypoints + final_path_waypoints

# Create function to generate path way 
def generate_random_path():
    """Generate a path with random middle waypoints (3-8) between fixed start/end"""
    # Randomly select 3-8 waypoints from the middle path
    num_waypoints = random.randint(CASO_MENOR, CASO_MAYOR)
    selected_waypoints = random.sample(middle_path_waypoints, num_waypoints)
    
    # Combine with fixed start and end points, points of interest and exit
    full_path = selected_waypoints + [random.choice(final_path_waypoints)]
    
    # Intialize the whole path way
    final_path = initial_path_waypoints
        
    # Make a loop to make the path way smoother for the agente
    i = 0
    for point in full_path:
        # Select each value of the actual objective
        row = point[0]
        column = point[1]
        
        # We calculate the exit of the person in the hallway
        if final_path[-1][0] > midle_value:
            final_path = final_path +  [np.array([1625, final_path[-1][1]])]
        else:
            final_path = final_path + [np.array([275, final_path[-1][1]])]
        
        # We calcule the entry to the hallway
        # The row value is the last value of the list final_path
        final_path = final_path + [np.array([final_path[-1][0], column])]
        
        # We add the actual objective
        final_path = final_path + [point]
        
        # We iterate to next objective        
        i = i + 1
    
    return final_path

# To check for the best position in the checkout
def best_slot(checkout_map):
    # Count occupied slots per line
    occupancy = {
        line: sum(1 for x, y, pid in slots if pid != -1)
        for line, slots in checkout_map.items()
    }
    # Pick the least-crowded line
    best_line = min(occupancy, key=occupancy.get)
    # Gather free slot indices
    free_idxs = [
        idx for idx, (_x, _y, pid) in enumerate(checkout_map[best_line])
        if pid == -1
    ]
    if not free_idxs:
        print("All queues are full.")
        return
    # Pick the last free slot
    idx = min(free_idxs)
    x, y, _ = checkout_map[best_line][idx]
    
    return best_line, [x, y]

# To register a person in the position
def register_person(checkout_map, person_id, coords):
    """
    Finds the slot matching coords [x, y] in checkout_map and sets its pid to person_id.
    Returns (line_key, slot_index) if successful, or (None, None) if coords not found.
    """
    x_target, y_target = coords
    for line, slots in checkout_map.items():
        for idx, (x, y, pid) in enumerate(slots):
            if x == x_target and y == y_target:
                checkout_map[line][idx][2] = person_id
                return line, idx
    return None, None

# When a person gets done
def unregister_person(checkout_map, person_id):
    """
    Finds the slot where pid == person_id in checkout_map,
    sets that slot’s pid back to -1, and returns (line_key, slot_index).
    If the person_id is not found, returns (None, None).
    """
    for line, slots in checkout_map.items():
        for idx, (x, y, pid) in enumerate(slots):
            if pid == person_id:
                checkout_map[line][idx][2] = -1
                return line, idx
    return None, None

FINAL_Y = 850  # last position y for all lines

def find_line_and_idx_by_coords(checkout_map, coords):
    x_t, y_t = coords
    for line, slots in checkout_map.items():
        for i, (x, y, _) in enumerate(slots):
            if x == x_t and y == y_t:
                return line, i
    return None, None

def try_advance_in_line(checkout_map, person_id, coords):
    """
    If the next slot towards the cashier is free, move the person forward.
    Returns the (possibly updated) coords they should target next.
    """
    line, idx = find_line_and_idx_by_coords(checkout_map, coords)
    if line is None:
        return coords  # not found; do nothing

    slots = checkout_map[line]
    last_idx = len(slots) - 1
    if idx < last_idx and slots[idx + 1][2] == -1:
        # Move forward one slot
        unregister_person(checkout_map, person_id)
        next_coords = slots[idx + 1][:2]
        register_person(checkout_map, person_id, next_coords)
        return next_coords
    return coords  # cannot advance yet

class Person:
    _id_counter = 1  # class‐level counter
    # Variables iniciales de la clase Person
    def __init__(self, x, y, path, infected=False, asymptomatic = False):
        # assign unique ID and increment counter
        self.id = Person._id_counter
        Person._id_counter += 1
        # Position variables
        self.position = np.array([x, y], dtype=float) # Actual position of the person
        self.velocity = np.random.rand(2) * MAX_SPEED # actual velocity of the person
        self.radius = PERSON_RADIUS # 
        self.path_index = 0 # Index of the path that is actually following
        self.path_threshold = 30  # Distance to consider waypoint reached
        self.active = True # True before checkout process
        self.path = path  # Each person gets its own path
        self.pulse_phase = 0  # Para el efecto de pulso
        # Checkout logic
        self.checkout = False # True in checkout process
        self.checkout_coords = [0, 0] # Position in the checkout logic
        # Logic infection
        self.infected = infected
        self.asymptomatic = asymptomatic
        if self.asymptomatic:
            self.base_color = YELLOW
        elif self.infected:
            self.base_color = RED
        else:
            self.base_color = BLUE
        # Add waiting-related attributes
        self.waiting = False
        self.wait_time = random.uniform(*WALK_WAIT_RANGE)  # Wait between 0.5-2 seconds        
        self.checkout_wait_time = random.uniform(*CHECKOUT_WAIT_RANGE)
        self.wait_start_time = 0
            
    def follow_path(self, sim_time):
        # Normal path process
        if self.path_index < len(self.path):
            target = self.path[self.path_index]
            direction = target - self.position
            distance = np.linalg.norm(direction)
            
            # Definir los puntos donde NO debe esperar
            x_value = self.path[self.path_index][0]
            is_special_point = x_value in [275, 1625]

            if distance < self.path_threshold:
                # Solo espera en waypoints intermedios, que NO son especiales
                if (
                    2 < self.path_index < len(self.path) - 1 and  # waypoints intermedios
                    not self.waiting and
                    not is_special_point
                ):
                    # Iniciar espera
                    self.waiting = True
                    self.wait_start_time = sim_time
                    self.velocity = np.zeros(2)  # Detenerse
                elif self.waiting:
                    # Checar si terminó de esperar
                    if sim_time - self.wait_start_time > self.wait_time:
                        self.waiting = False
                        self.path_index += 1
                else:
                    # Para los puntos especiales, los dos primeros y el último: avanzar sin esperar
                    self.path_index += 1

                # Si llegó al final, desactiva el camino normal y activa checkout process
                if self.path_index >= len(self.path):
                    self.active = False
                    self.checkout = True
                    # Calculate the best new position for the person
                    line, self.checkout_coords = best_slot(checkout_map)
                    # Register in the chekout map the position of the person
                    line, idx = register_person(checkout_map, person_id=self.id, coords=self.checkout_coords )
            elif distance > 0 and not self.waiting:
                direction_normalized = direction / distance
                self.velocity += direction_normalized * 0.1
        # Checkout process
        else:
            target = np.array(self.checkout_coords)
            direction = target - self.position
            distance = np.linalg.norm(direction)

            if distance < self.path_threshold:
                line, idx = find_line_and_idx_by_coords(checkout_map, self.checkout_coords)
                if line is None:
                    # safety: re-acquire a spot
                    line, self.checkout_coords = best_slot(checkout_map)
                    register_person(checkout_map, self.id, self.checkout_coords)
                    return

                slots = checkout_map[line]
                last_idx = len(slots) - 1
                at_final_slot = (idx == last_idx) or (self.checkout_coords[1] == FINAL_Y)

                if at_final_slot:
                    # Wait only at the final slot for checkout duration
                    if not self.waiting:
                        self.waiting = True
                        self.wait_start_time = sim_time
                        self.velocity = np.zeros(2)
                    else:
                        if sim_time - self.wait_start_time > self.checkout_wait_time:
                            # Done paying; leave queue
                            self.waiting = False
                            self.checkout = False
                            unregister_person(checkout_map, self.id)
                else:
                    # INTERMEDIATE SLOTS: stand still here, do not roam
                    self.velocity = np.zeros(2)
                    self.waiting = True  # freeze at current slot

                    # If the next slot toward cashier is free, advance
                    if slots[idx + 1][2] == -1:
                        self.waiting = False  # allow movement toward the next slot
                        unregister_person(checkout_map, self.id)
                        next_coords = slots[idx + 1][:2]
                        register_person(checkout_map, self.id, next_coords)
                        self.checkout_coords = next_coords
                        # (movement vector will be set below on the next tick)
            elif distance > 0 and not self.waiting:
                # move toward current checkout target
                direction_normalized = direction / distance
                self.velocity += direction_normalized * 0.1

    def update(self, people, walls, sim_time):
        if not self.active and not self.checkout:
            return

        # Always update intent/targets first
        self.follow_path(sim_time)

        # If in checkout:
        if self.checkout:
            # If waiting at a slot, don't move
            if self.waiting:
                return
            # Only basic movement (no separation/wall forces)
            speed = np.linalg.norm(self.velocity)
            if speed > MAX_SPEED:
                self.velocity = (self.velocity / speed) * MAX_SPEED
            self.position += self.velocity
            self.position = np.clip(self.position, [0, 0], [WIDTH, HEIGHT])
            return

        # Normal world movement (not in checkout)
        if self.waiting:
            return
            
        # Rest of the update method remains the same...
        separation = np.zeros(2)
        for other in people:
            if other != self and other.active:
                dist = np.linalg.norm(self.position - other.position)
                if dist < SEPARATION_RADIUS:
                    separation += (self.position - other.position) / (dist + 0.0001)
        
        for wall in walls:
            dist_to_wall = self.distance_to_wall(wall)
            if dist_to_wall < WALL_MARGIN:
                separation += (self.position - wall[:2]) / (dist_to_wall + 0.0001)
        
        self.velocity += separation * 0.1
        speed = np.linalg.norm(self.velocity)
        if speed > MAX_SPEED:
            self.velocity = (self.velocity / speed) * MAX_SPEED
        self.position += self.velocity
        self.position = np.clip(self.position, [0, 0], [WIDTH, HEIGHT])
    
    def distance_to_wall(self, wall):
        x1, y1, x2, y2 = wall
        px, py = self.position
        line_length = np.linalg.norm(np.array([x2 - x1, y2 - y1]))
        if line_length == 0:
            return np.linalg.norm(self.position - np.array([x1, y1]))
        t = max(0, min(1, np.dot([px - x1, py - y1], [x2 - x1, y2 - y1]) / line_length**2))
        closest_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
        return np.linalg.norm(self.position - closest_point)
    
    def draw(self, screen, sim_time):
        if not self.active and not self.checkout:
            return

        if self.infected or self.asymptomatic:
            # Tiempo animado en segundos
            time = sim_time # más suave que *0.005

            # Pulso de tamaño usando seno (entre 2.5x y 3.5x del radio)
            pulse_scale = 4 + 4 * math.sin(time)
            outer_radius = max(1, int(self.radius * pulse_scale))

            # Color pulsante (entre rosa claro y rojo intenso)
            pulse_color = (
                255,
                int(100 + 50 * math.sin(time)),  # valor entre 50 y 150
                int(100 + 50 * math.sin(time))
            )

            # Dibuja círculo externo
            pygame.draw.circle(screen, pulse_color, self.position, outer_radius, width=2)

        # Dibuja la persona (círculo base)
        pygame.draw.circle(screen, self.base_color, self.position, self.radius)

# Define passable walls (blue - can walk through)
passable_walls = [
    #[800, 400, 1000, 400],   # Passage between horizontal walls
    #[900, 400, 900, 600]     # Passage between vertical walls
]

# Initialize empty people list
# Spawn timer

def draw_logic():
    # Clear screen
    screen.fill(WHITE)

    # Draw path waypoints and connecting lines
    for i, point in enumerate(path_waypoints):
        pygame.draw.circle(screen, GREEN, point.astype(int), 5)
        if i > 0:
            pygame.draw.line(
                screen, (200, 200, 200),
                path_waypoints[i - 1],
                point,
                2
            )

    # Draw checkout slots
    for line_name, slots in checkout_map.items():
        for slot in slots:
            point = np.array([slot[0], slot[1]])
            pygame.draw.circle(screen, YELLOW, point.astype(int), 8)

    # Draw walls
    for wall in walls:
        pygame.draw.line(screen, BLACK, (wall[0], wall[1]), (wall[2], wall[3]), 5)
    for wall in passable_walls:
        pygame.draw.line(screen, RED, (wall[0], wall[1]), (wall[2], wall[3]), 3)

def reset_checkout_map():
    for fila in checkout_map:
        for posicion in checkout_map[fila]:
            posicion[2] = -1
  
#  Main simulation loop: run the simulation NUM_RUNS times
# ---------------------------------------------------------------------
NUM_RUNS = 2
run_durations = []  # <-- collect per-run durations (in seconds)
FAST_MODE = False
TARGET_FPS = 60
DT = 1.0 / TARGET_FPS  # fixed sim step (seconds)
SPAWN_INTERVAL_S = SPAWN_INTERVAL / 1000.0  # convert ms -> s
random.seed(42)
np.random.seed(42)
  
# Pygame setup
pygame.init()
if FAST_MODE:
    # draw into an off-screen surface
    screen = pygame.Surface((WIDTH, HEIGHT))
    #os.environ["SDL_VIDEODRIVER"] = "dummy"  # or use pygame.HIDDEN flag alternative
else:
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
font = pygame.font.SysFont('Arial', 20)

for sim_run in range(NUM_RUNS):
    # Initial setupo
    last_spawn_time = 0.0
    sim_time = 0.0
    running = True
    reset_checkout_map()
    people = []
    people.clear()            # drop references to all Person objects
    Person._id_counter = 1    # optional: restart IDs next run
    # Print an action time to time, to debug
    ID_DUMP_INTERVAL = 20.0  # seconds
    next_id_dump = ID_DUMP_INTERVAL

    while running:
        # --- time step ---
        if FAST_MODE:
            dt = DT                 # run as fast as the CPU allows
        else:
            dt = DT                 # run as fast as the CPU allows
            #dt = clock.tick(TARGET_FPS) / 1000.0  # real-time pacing
        sim_time += dt
        
        # --- events (keep tiny to avoid OS “not responding”) ---
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit(); sys.exit()

        # --- spawns based on SIM time, not wall time ---
        if (sim_time - last_spawn_time) > (SPAWN_INTERVAL / 1000.0) and len(people) < NUM_PEOPLE:
            group_path = generate_random_path()
            spawn_count = min(GROUP_SIZE, NUM_PEOPLE - len(people))
            for _ in range(spawn_count):
                infected = random.random() < INFECTION_RATE
                asymptomatic = infected and random.random() < ASYMPTOMATIC_RATE
                print("Infectado")
                print(infected)
                print("Asínstomatico")
                print(asymptomatic)
                x = random.randint(SPAWN_AREA[0], SPAWN_AREA[1])
                y = random.randint(SPAWN_AREA[2], SPAWN_AREA[3])
                people.append(Person(x, y, group_path, infected, asymptomatic))
            last_spawn_time = sim_time
            
        # Print an action time to time, to debug
        while sim_time >= next_id_dump:
            all_ids = [p.id for p in people]  # or only active: [p.id for p in people if p.active or p.checkout]
            all_infected = [p.infected for p in people]  # or only active: [p.id for p in people if p.active or p.checkout]
            
            print(f"[t={next_id_dump:.0f}s] People IDs: {all_ids}")
            print(f"[t={next_id_dump:.0f}s] People infected: {all_infected}")
            
            next_id_dump += ID_DUMP_INTERVAL

        # --- update ---
        active_count = 0
        for person in people:
            person.update(people, walls, sim_time)
            if person.active or person.checkout:
                active_count += 1

        # --- draw only if NOT fast mode ---
        if not FAST_MODE:
            draw_logic()
            for person in people:
                person.draw(screen, sim_time)
            status_text = (f"Run {sim_run+1}/{NUM_RUNS}  "
                           f"Active: {active_count}/{NUM_PEOPLE}  "
                           f"t={sim_time:.1f}s")
            screen.blit(font.render(status_text, True, BLACK), (10, 10))
            pygame.display.flip()

        # --- finish condition ---
        if active_count == 0 and len(people) == NUM_PEOPLE:
            print(f"Simulation {sim_run+1} finished in {sim_time:.2f} s")
            running = False

pygame.quit()
sys.exit()

Infectado
True
Asínstomatico
False
Infectado
True
Asínstomatico
False
Infectado
True
Asínstomatico
True
Infectado
True
Asínstomatico
False
Infectado
False
Asínstomatico
False
Infectado
True
Asínstomatico
False
Infectado
True
Asínstomatico
False
Infectado
False
Asínstomatico
False
Infectado
True
Asínstomatico
False
Infectado
True
Asínstomatico
True
Infectado
False
Asínstomatico
False
Infectado
True
Asínstomatico
False
Infectado
True
Asínstomatico
False
Infectado
False
Asínstomatico
False
Infectado
True
Asínstomatico
False
Infectado
True
Asínstomatico
True
Infectado
True
Asínstomatico
False
Infectado
False
Asínstomatico
False
Infectado
True
Asínstomatico
True
Infectado
True
Asínstomatico
False
Infectado
False
Asínstomatico
False
Infectado
True
Asínstomatico
False
Infectado
False
Asínstomatico
False
Infectado
True
Asínstomatico
False
Infectado
True
Asínstomatico
False
Infectado
True
Asínstomatico
True
Infectado
True
Asínstomatico
False
Infectado
True
Asínstomatico
True
Infectado
False
Así

SystemExit: 

In [ ]:
checkout_map = {
    "F1": [
        [350, 700, -1], 
        [350, 750, 5],        
        [350, 800, 1],        
        [350, 850, 5],        
    ],
    "F2": [
        [600, 700, -1], 
        [600, 750, 3],        
        [600, 800, 2],        
        [600, 850, 2],        
    ],
    "F3": [
        [850, 700, -1], 
        [850, 750, 8],        
        [850, 800, 17],        
        [850, 850, 7],        
    ],
    "F4": [
        [1100, 700, -1], 
        [1100, 750, -1],        
        [1100, 800, -1],        
        [1100, 850, 9],        
    ],
    "F5": [
        [1350, 700, -1], 
        [1350, 750, -1],        
        [1350, 800, 19],        
        [1350, 850, 10],        
    ],
    "F6": [
        [1600, 700, -1], 
        [1600, 750, -1],        
        [1600, 800, 5],        
        [1600, 850, 11],        
    ],
}

def best_slot(checkout_map):
    # Count occupied slots per line
    occupancy = {
        line: sum(1 for x, y, pid in slots if pid != -1)
        for line, slots in checkout_map.items()
    }
    # Pick the least-crowded line
    best_line = min(occupancy, key=occupancy.get)
    # Gather free slot indices
    free_idxs = [
        idx for idx, (_x, _y, pid) in enumerate(checkout_map[best_line])
        if pid == -1
    ]
    if not free_idxs:
        print("All queues are full.")
        return
    # Pick the last free slot
    idx = max(free_idxs)
    x, y, _ = checkout_map[best_line][idx]
    
    return best_line, [x, y]

def register_person(checkout_map, person_id, coords):
    """
    Finds the slot matching coords [x, y] in checkout_map and sets its pid to person_id.
    Returns (line_key, slot_index) if successful, or (None, None) if coords not found.
    """
    x_target, y_target = coords
    for line, slots in checkout_map.items():
        for idx, (x, y, pid) in enumerate(slots):
            if x == x_target and y == y_target:
                checkout_map[line][idx][2] = person_id
                return line, idx
    return None, None

def unregister_person(checkout_map, person_id):
    """
    Finds the slot where pid == person_id in checkout_map,
    sets that slot’s pid back to -1, and returns (line_key, slot_index).
    If the person_id is not found, returns (None, None).
    """
    for line, slots in checkout_map.items():
        for idx, (x, y, pid) in enumerate(slots):
            if pid == person_id:
                checkout_map[line][idx][2] = -1
                return line, idx
    return None, None

for fila in checkout_map:
    print(fila, ":")
    for posicion in checkout_map[fila]:
        print(np.array([posicion[0], posicion[1]]))
        


{'F1': [[350, 700, -1], [350, 750, 5], [350, 800, 1], [350, 850, 5]], 'F2': [[600, 700, -1], [600, 750, 3], [600, 800, 2], [600, 850, 2]], 'F3': [[850, 700, -1], [850, 750, 8], [850, 800, 17], [850, 850, 7]], 'F4': [[1100, 700, -1], [1100, 750, -1], [1100, 800, -1], [1100, 850, 9]], 'F5': [[1350, 700, -1], [1350, 750, -1], [1350, 800, 19], [1350, 850, 10]], 'F6': [[1600, 700, -1], [1600, 750, -1], [1600, 800, 5], [1600, 850, 11]]}
F1 :
[350 700]
[350 750]
[350 800]
[350 850]
F2 :
[600 700]
[600 750]
[600 800]
[600 850]
F3 :
[850 700]
[850 750]
[850 800]
[850 850]
F4 :
[1100  700]
[1100  750]
[1100  800]
[1100  850]
F5 :
[1350  700]
[1350  750]
[1350  800]
[1350  850]
F6 :
[1600  700]
[1600  750]
[1600  800]
[1600  850]


In [8]:
def best_slot(checkout_map):
    # Count occupied slots per line
    occupancy = {
        line: sum(1 for x, y, pid in slots if pid != -1)
        for line, slots in checkout_map.items()
    }
    # Pick the least-crowded line
    best_line = min(occupancy, key=occupancy.get)
    # Gather free slot indices
    free_idxs = [
        idx for idx, (_x, _y, pid) in enumerate(checkout_map[best_line])
        if pid == -1
    ]
    print(free_idxs)
    if not free_idxs:
        print("All queues are full.")
        return
    # Pick the last free slot
    idx = min(free_idxs)
    print(idx)
    x, y, _ = checkout_map[best_line][idx]
    
    return best_line, [x, y]

checkout_map = {
    "F1": [
        [350, 700, -1],
        [350, 750, 5],
        [350, 800, 1],
        [350, 850, 5],
    ],
    "F2": [
        [600, 700, -1],
        [600, 750, 3],
        [600, 800, 2],
        [600, 850, -1],
    ],
}

for fila in checkout_map:
    for posicion in checkout_map[fila]:
        posicion[2] = -1
print(checkout_map)
        


{'F1': [[350, 700, -1], [350, 750, -1], [350, 800, -1], [350, 850, -1]], 'F2': [[600, 700, -1], [600, 750, -1], [600, 800, -1], [600, 850, -1]]}


In [ ]:
checkout_map = {
        "F1": [
            [350, 650, -1], 
            [350, 700, -1], 
            [350, 750, -1],        
            [350, 800, -1],        
            [350, 850, -1],        
        ],
        "F2": [
            [470, 650, -1], 
            [470, 700, -1], 
            [470, 750, -1],        
            [470, 800, -1],        
            [470, 850, -1],        
        ],
        "F3": [
            [600, 650, -1], 
            [600, 700, -1], 
            [600, 750, -1],        
            [600, 800, -1],        
            [600, 850, -1],        
        ],
        "F4": [
            [720, 650, -1], 
            [720, 700, -1], 
            [720, 750, -1],        
            [720, 800, -1],        
            [720, 850, -1],        
        ],
        "F5": [
            [850, 650, -1], 
            [850, 700, -1], 
            [850, 750, -1],        
            [850, 800, -1],        
            [850, 850, -1],        
        ],
        "F6": [
            [970, 650, -1], 
            [970, 700, -1], 
            [970, 750, -1],        
            [970, 800, -1],        
            [970, 850, -1],        
        ],
        "F7": [
            [1100, 650, -1], 
            [1100, 700, -1], 
            [1100, 750, -1],        
            [1100, 800, -1],        
            [1100, 850, -1],        
        ],
        "F8": [
            [1220, 650, -1], 
            [1220, 700, -1], 
            [1220, 750, -1],        
            [1220, 800, -1],        
            [1220, 850, -1],        
        ],
        "F9": [
            [1350, 650, -1], 
            [1350, 700, -1], 
            [1350, 750, -1],        
            [1350, 800, -1],        
            [1350, 850, -1],        
        ],
        "F10": [
            [1470, 650, -1], 
            [1470, 700, -1], 
            [1470, 750, -1],        
            [1470, 800, -1],        
            [1470, 850, -1],        
        ],
        "F11": [
            [1600, 650, -1], 
            [1600, 700, -1], 
            [1600, 750, -1],        
            [1600, 800, -1],        
            [1600, 850, -1],        
        ],
    }